In [1]:
!pip install datasets

   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ------------------------------------ --- 2.6/2.9 MB 13.5 MB/s eta 0:00:01
   ---------------------------------------- 2.9/2.9 MB 13.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/28.0 MB ? eta -:--:--
   ---- ----------------------------------- 3.4/28.0 MB 16.0 MB/s eta 0:00:02
   -------- ------------------------------- 6.3/28.0 MB 15.4 MB/s eta 0:00:02
   -------------- ------------------------- 10.0/28.0 MB 16.0 MB/s eta 0:00:02
   ------------------ --------------------- 13.1/28.0 MB 15.8 MB/s eta 0:00:01
   ----------------------- ---------------- 16.8/28.0 MB 16.3 MB/s eta 0:00:01
   ----------------------------- ---------- 20.4/28.0 MB 16.4 MB/s eta 0:00:01
   ---------------------------------- ----- 23.9/28.0 MB 16.7 MB/s eta 0:00:01
   -------------------------------------- - 27.0/28.0 MB 16.4 MB/s eta 0:00:01
   ---------------------------------------- 28.0/28.0 MB 15.9 MB/s eta 0:00:0


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from datasets import load_dataset

ds = load_dataset("mariosyahirhalimm/loan_prediction_dataset")
df = ds["train"].to_pandas()

c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User\.cache\huggingface\hub\datasets--mariosyahirhalimm--loan_prediction_dataset. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to 

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import joblib

In [4]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [5]:
df.shape

(614, 13)

In [6]:
df.tail()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y
613,LP002990,Female,No,0,Graduate,Yes,4583,0.0,133.0,360.0,0.0,Semiurban,N


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [8]:
# Handling Missing Value
df.isnull().sum()

,0
Loan_ID,0
Gender,13
Married,3
Dependents,15
Education,0
Self_Employed,32
ApplicantIncome,0
CoapplicantIncome,0
LoanAmount,22
Loan_Amount_Term,14


In [9]:
df.isnull().mean()*100

,0
Loan_ID,0.000000
Gender,2.117264
Married,0.488599
Dependents,2.442997
Education,0.000000
Self_Employed,5.211726
ApplicantIncome,0.000000
CoapplicantIncome,0.000000
LoanAmount,3.583062
Loan_Amount_Term,2.280130


In [10]:
df = df.drop('Loan_ID', axis =1)

In [11]:
df.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [12]:
df = df.dropna(subset = ['Gender', 'Dependents', 'Loan_Amount_Term'])

In [13]:
df.shape

(573, 12)

In [14]:
df.isnull().sum()

,0
Gender,0
Married,0
Dependents,0
Education,0
Self_Employed,31
ApplicantIncome,0
CoapplicantIncome,0
LoanAmount,20
Loan_Amount_Term,0
Credit_History,49


In [15]:
df['Self_Employed'].unique()

array(['No', 'Yes', None], dtype=object)

In [16]:
df['Self_Employed'].mode()[0]

'No'

In [17]:
df['LoanAmount'].unique()

array([ nan, 128.,  66., 120., 141., 267.,  95., 158., 168., 349.,  70.,
       109., 200., 114.,  17., 125., 100.,  76., 133., 104., 315., 116.,
       151., 191., 122., 110.,  35., 201.,  74., 106., 320., 144., 184.,
        80.,  47.,  75., 134.,  44., 112., 286.,  97.,  96., 135., 180.,
        99., 165., 258., 126., 312., 136., 172.,  81., 187., 113., 176.,
       130., 111., 167., 265.,  50., 210., 175., 131., 188.,  25., 137.,
       115., 225., 216.,  94., 139., 118., 185., 154.,  85., 259., 194.,
        93., 160., 182., 650., 102., 290.,  84.,  88., 242., 129.,  30.,
       152., 244., 600., 255.,  98., 275., 121.,  63.,  87., 101., 495.,
        67.,  73., 260., 108.,  58.,  48., 164., 170.,  83.,  90., 166.,
       124.,  55.,  59., 127., 214., 240.,  72.,  60.,  42., 280., 140.,
       155., 123., 138., 279., 192., 304., 330., 150., 207., 436.,  78.,
        54.,  89., 132., 480.,  56., 300., 376., 117., 490.,  71., 173.,
        46., 228., 308., 105., 236., 570., 380., 29

In [18]:
df['LoanAmount'].median()

128.0

In [19]:
df['Credit_History'].unique()

array([ 1.,  0., nan])

In [20]:
df['Credit_History'].mode()[0]

np.float64(1.0)

In [21]:
df['Self_Employed'].fillna(df['Self_Employed'].mode()[0], inplace=True)
df['LoanAmount'].fillna(df['LoanAmount'].median(), inplace=True)
df['Credit_History'].fillna(df['Credit_History'].mode()[0], inplace=True)

/tmp/ipython-input-3290767669.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Self_Employed'].fillna(df['Self_Employed'].mode()[0], inplace=True)
/tmp/ipython-input-3290767669.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value,

In [22]:
df.isnull().sum()

,0
Gender,0
Married,0
Dependents,0
Education,0
Self_Employed,0
ApplicantIncome,0
CoapplicantIncome,0
LoanAmount,0
Loan_Amount_Term,0
Credit_History,0


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 573 entries, 0 to 613
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gender             573 non-null    object 
 1   Married            573 non-null    object 
 2   Dependents         573 non-null    object 
 3   Education          573 non-null    object 
 4   Self_Employed      573 non-null    object 
 5   ApplicantIncome    573 non-null    int64  
 6   CoapplicantIncome  573 non-null    float64
 7   LoanAmount         573 non-null    float64
 8   Loan_Amount_Term   573 non-null    float64
 9   Credit_History     573 non-null    float64
 10  Property_Area      573 non-null    object 
 11  Loan_Status        573 non-null    object 
dtypes: float64(4), int64(1), object(7)
memory usage: 58.2+ KB


In [24]:
df['Gender'].unique()

array(['Male', 'Female'], dtype=object)

In [25]:
df['Dependents'].unique()

array(['0', '1', '2', '3+'], dtype=object)

In [26]:
df['Dependents'].replace('3+', '4', inplace=True)

/tmp/ipython-input-2245831984.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Dependents'].replace('3+', '4', inplace=True)


In [27]:
df['Dependents'].replace('4', '3', inplace=True)

/tmp/ipython-input-3363379990.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Dependents'].replace('4', '3', inplace=True)


In [28]:
df['Dependents'].unique()

array(['0', '1', '2', '3'], dtype=object)

In [29]:
df['Married'].unique()

array(['No', 'Yes'], dtype=object)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 573 entries, 0 to 613
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gender             573 non-null    object 
 1   Married            573 non-null    object 
 2   Dependents         573 non-null    object 
 3   Education          573 non-null    object 
 4   Self_Employed      573 non-null    object 
 5   ApplicantIncome    573 non-null    int64  
 6   CoapplicantIncome  573 non-null    float64
 7   LoanAmount         573 non-null    float64
 8   Loan_Amount_Term   573 non-null    float64
 9   Credit_History     573 non-null    float64
 10  Property_Area      573 non-null    object 
 11  Loan_Status        573 non-null    object 
dtypes: float64(4), int64(1), object(7)
memory usage: 58.2+ KB


In [31]:
encoding = {
    'Gender': {'Male': 1, 'Female': 0},
    'Married': {'Yes': 1, 'No': 0},
    'Dependents': {'0': 0, '1': 1, '2': 2, '3': 3},
    'Education': {'Graduate': 1, 'Not Graduate': 0},
    'Self_Employed': {'Yes': 1, 'No': 0},
    'Property_Area': {'Rural': 0, 'Semiurban': 1, 'Urban': 2},
    'Loan_Status': {'Y': 1, 'N': 0}
}

In [32]:
df.replace(encoding, inplace= True)

/tmp/ipython-input-1044600927.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(encoding, inplace= True)


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 573 entries, 0 to 613
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gender             573 non-null    int64  
 1   Married            573 non-null    int64  
 2   Dependents         573 non-null    int64  
 3   Education          573 non-null    int64  
 4   Self_Employed      573 non-null    int64  
 5   ApplicantIncome    573 non-null    int64  
 6   CoapplicantIncome  573 non-null    float64
 7   LoanAmount         573 non-null    float64
 8   Loan_Amount_Term   573 non-null    float64
 9   Credit_History     573 non-null    float64
 10  Property_Area      573 non-null    int64  
 11  Loan_Status        573 non-null    int64  
dtypes: float64(4), int64(8)
memory usage: 58.2 KB


In [34]:
x = df.drop('Loan_Status', axis = 1) #independent variable
y = df['Loan_Status'] #dependent variable - model learn to predict y based on the values in x

In [35]:
df.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,1,0,0,1,0,5849,0.0,128.0,360.0,1.0,2,1
1,1,1,1,1,0,4583,1508.0,128.0,360.0,1.0,0,0
2,1,1,0,1,1,3000,0.0,66.0,360.0,1.0,2,1
3,1,1,0,0,0,2583,2358.0,120.0,360.0,1.0,2,1
4,1,0,0,1,0,6000,0.0,141.0,360.0,1.0,2,1


In [36]:
num_cols = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term'] # Z-score normalization
scaler = StandardScaler()
x[num_cols] = scaler.fit_transform(x[num_cols])

In [37]:
x.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,1,0,0,1,0,0.088156,-0.547390,-0.210425,0.281358,1.0,2
1,1,1,1,1,0,-0.126269,-0.044475,-0.210425,0.281358,1.0,0
2,1,1,0,1,1,-0.394385,-0.547390,-0.961362,0.281358,1.0,2
3,1,1,0,0,0,-0.465013,0.238998,-0.307320,0.281358,1.0,2
4,1,0,0,1,0,0.113731,-0.547390,-0.052971,0.281358,1.0,2


In [38]:
def evaluate_model(model):
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)
  model.fit(x_train, y_train)
  y_pred = model.predict(x_test)
  accuracy = accuracy_score(y_test, y_pred)
  cross_val = cross_val_score(model, x, y, cv=5)
  avg_cross_val = np.mean(cross_val)
  print(f"{model.__class__.__name__} - Accuracy : {accuracy: .2f}, Cross-Val-Score: {avg_cross_val: .2f}")
  return avg_cross_val

In [39]:
models = {
    LogisticRegression(),
    svm.SVC(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    GradientBoostingClassifier(),
}

In [40]:
model_score = {model.__class__.__name__:evaluate_model(model) for model in models}

LogisticRegression - Accuracy :  0.81, Cross-Val-Score:  0.80
RandomForestClassifier - Accuracy :  0.79, Cross-Val-Score:  0.78
DecisionTreeClassifier - Accuracy :  0.69, Cross-Val-Score:  0.71
GradientBoostingClassifier - Accuracy :  0.77, Cross-Val-Score:  0.78
SVC - Accuracy :  0.80, Cross-Val-Score:  0.79


In [41]:
def tune_model(model, param_grid):
  tuner = RandomizedSearchCV(model, param_grid, cv = 5, n_iter = 20, verbose = True, random_state = 42)
  tuner.fit(x, y)
  print(f"Best Score for {model.__class__.__name__}: {tuner.best_score_: .2f}")
  print(f"Best Parameter for {model.__class__.__name__}: {tuner.best_params_}")
  return tuner.best_estimator_

In [42]:
log_reg_grid = {'C': np.logspace(-4, 4, 20), "solver": ["liblinear"]}
svc_grid = {'C': [0.25, 0.50, 0.75, 1], "kernel": ['linear']}

rf_grid = {
    'n_estimators': np.arange(10, 1000, 10),
    'max_features': ['log2', 'sqrt'],
    'max_depth': [None, 3, 5, 10, 20, 30],
    'min_samples_split': [2, 5, 20, 50, 100],
    'min_samples_leaf': [1, 2, 5, 10]

}

In [43]:
best_log_reg = tune_model(LogisticRegression(), log_reg_grid)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Score for LogisticRegression:  0.80
Best Parameter for LogisticRegression: {'solver': 'liblinear', 'C': np.float64(0.23357214690901212)}


In [44]:
_svc_reg = tune_model(svm.SVC(), svc_grid)

/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 4 is smaller than n_iter=20. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best Score for SVC:  0.80
Best Parameter for SVC: {'kernel': 'linear', 'C': 0.25}


In [46]:
best_rf = tune_model(RandomForestClassifier(), rf_grid)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Score for RandomForestClassifier:  0.80
Best Parameter for RandomForestClassifier: {'n_estimators': np.int64(690), 'min_samples_split': 50, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': None}


In [47]:
final_model = best_log_reg

In [48]:
joblib.dump(final_model, 'loan_status_predictor')

['loan_status_predictor']